# Text Classification using Logistic Regression (Week 3)

This lab is prepared with the tutorial, Deep Learning with PyTorch (https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html)



Deep Learning Building Blocks: Affine maps, non-linearities and objectives
==========================================================================

Deep learning consists of composing linearities with non-linearities in
clever ways. The introduction of non-linearities allows for powerful
models. In this section, we will play with these core components, make
up an objective function, and see how the model is trained.


Affine Maps:

One of the core workhorses of deep learning is the affine map, which is
a function $f(x)$ where

\begin{align}f(x) = Ax + b\end{align}

for a matrix $A$ and vectors $x, b$. The parameters to be
learned here are $A$ and $b$. Often, $b$ is refered to
as the *bias* term.


<b>PyTorch and most other deep learning frameworks do things a little
differently than traditional linear algebra. It maps the rows of the
input instead of the columns. That is, the $i$'th row of the
output below is the mapping of the $i$'th row of the input under
$A$, plus the bias term.</b> Look at the example below.




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
data = torch.randn(1, 5)  # data is 1x5.
print("input: \n", data)

input: 
 tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]])


In [3]:
lin = nn.Linear(5, 1)  # maps from R^5 to R^1; Applies a linear transformation to the incoming data with parameters A, b
                       # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

print("output: \n", lin(data))  # maps from 5 to 1 with parameters A, b; calculate Z score

output: 
 tensor([[0.0034]], grad_fn=<AddmmBackward0>)


In [4]:
lin.weight # W weights (initial values)

Parameter containing:
tensor([[ 0.1633, -0.1743, -0.0326, -0.0403,  0.0648]], requires_grad=True)

In [5]:
lin.bias # b bias (initial values)

Parameter containing:
tensor([-0.0018], requires_grad=True)

Non-Linearities:

First, note the following fact, which will explain why we need
non-linearities in the first place. Suppose we have two affine maps
$f(x) = Ax + b$ and $g(x) = Cx + d$. What is
$f(g(x))$?

\begin{align}f(g(x)) = A(Cx + d) + b = ACx + (Ad + b)\end{align}

$AC$ is a matrix and $Ad + b$ is a vector, so we see that
composing affine maps gives you an affine map.

From this, you can see that if you wanted your neural network to be long
chains of affine compositions, that this adds no new power to your model
than just doing a single affine map.

If we introduce non-linearities in between the affine layers, this is no
longer the case, and we can build much more powerful models.

There are a few core non-linearities.
$\tanh(x), \sigma(x), \text{ReLU}(x)$ are the most common. You are
probably wondering: "why these functions? I can think of plenty of other
non-linearities." The reason for this is that they have gradients that
are easy to compute, and computing gradients is essential for learning.
For example

\begin{align}\frac{d\sigma}{dx} = \sigma(x)(1 - \sigma(x))\end{align}

A quick note: although you may have learned some neural networks in your
intro to AI class where $\sigma(x)$ was the default non-linearity,
typically people shy away from it in practice. This is because the
gradient *vanishes* very quickly as the absolute value of the argument
grows. Small gradients means it is hard to learn. Most people default to
tanh or ReLU.




In [6]:
# In pytorch, most non-linearities are in torch.functional (we have it imported as F)
# Note that non-linearites typically don't have parameters like affine maps do.
# That is, they don't have weights that are updated during training.
data = lin(data)
print(data, "\n")
print("sigmoid(): \n", torch.sigmoid(data))
print("relu(): \n", F.relu(data))

tensor([[0.0034]], grad_fn=<AddmmBackward0>) 

sigmoid(): 
 tensor([[0.5008]], grad_fn=<SigmoidBackward0>)
relu(): 
 tensor([[0.0034]], grad_fn=<ReluBackward0>)


Objective Functions:

The objective function is the function that your network is being
trained to minimize (in which case it is often called a *loss function*
or *cost function*). This proceeds by first choosing a training
instance, running it through your neural network, and then computing the
loss of the output. The parameters of the model are then updated by
taking the derivative of the loss function. Intuitively, if your model
is completely confident in its answer, and its answer is wrong, your
loss will be high. If it is very confident in its answer, and its answer
is correct, the loss will be low.

The idea behind minimizing the loss function on your training examples
is that your network will hopefully generalize well and have small loss
on unseen examples in your dev set, test set, or in production. An
example loss function is the *negative log likelihood loss*, which is a
very common objective for multi-class classification. For supervised
multi-class classification, this means training the network to minimize
the negative log probability of the correct output (or equivalently,
maximize the log probability of the correct output).

目标函数：

目标函数是您的网络正在训练以最小化的函数（在这种情况下，它通常被称为损失函数或成本函数）。这个过程首先选择一个训练实例，将其输入到神经网络中，然后计算输出的损失。接着，模型的参数通过对损失函数求导来进行更新。

直观来说，如果您的模型对其答案完全自信，而它的答案是错误的，那么损失会很高。如果它对其答案非常自信，并且答案是正确的，那么损失会很低。

最小化训练示例上的损失函数的目的是让您的网络能够很好地泛化，并在开发集、测试集或实际应用中对未见过的示例具有较小的损失。

一个示例损失函数是负对数似然损失，这是多类别分类中非常常见的目标函数。在监督式多类别分类中，这意味着训练网络以最小化正确输出的负对数概率（或者等价地，最大化正确输出的对数概率）。



Optimization and Training
=========================

So what we can compute a loss function for an instance? What do we do
with that? We saw earlier that Tensors know how to compute gradients
with respect to the things that were used to compute it. Well,
since our loss is an Tensor, we can compute gradients with
respect to all of the parameters used to compute it! Then we can perform
standard gradient updates. Let $\theta$ be our parameters,
$L(\theta)$ the loss function, and $\eta$ a positive
learning rate. Then:

\begin{align}\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta)\end{align}

There are a huge collection of algorithms and active research in
attempting to do something more than just this vanilla gradient update.
Many attempt to vary the learning rate based on what is happening at
train time. You don't need to worry about what specifically these
algorithms are doing unless you are really interested. Torch provides
many in the torch.optim package, and they are all completely
transparent. Using the simplest gradient update is the same as the more
complicated algorithms. Trying different update algorithms and different
parameters for the update algorithms (like different initial learning
rates) is important in optimizing your network's performance. Often,
just replacing vanilla SGD with an optimizer like Adam or RMSProp will
boost performance noticably.


**优化和训练**

那么，我们可以为一个实例计算损失函数，那么接下来该做什么呢？我们之前看到，张量（Tensors）知道如何计算相对于用于计算它的变量的梯度。由于我们的损失是一个张量，我们可以计算相对于所有用于计算它的参数的梯度！然后，我们可以执行标准的梯度更新。设 \(\theta\) 为我们的参数，\(\mathcal{L}(\theta)\) 为损失函数，\(\eta\) 为一个正的学习率。那么：

\begin{align}\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta)\end{align}

有大量的算法和活跃的研究，致力于做比这种基本的梯度更新更多的优化。许多算法尝试根据训练过程中发生的情况调整学习率。除非您真的感兴趣，否则无需担心这些算法具体在做什么。Torch 在 `torch.optim` 包中提供了许多优化算法，并且它们都是完全透明的。使用最简单的梯度更新与更复杂的算法本质上是相同的。尝试不同的更新算法以及不同的更新参数（例如不同的学习率）对优化神经网络的性能非常重要。通常，仅仅用 Adam 或 RMSProp 这样的优化器替换基本的随机梯度下降（SGD），就能显著提升性能。


Creating Network Components in PyTorch
======================================

Before we move on to our focus on NLP, lets do an annotated example of
building a network in PyTorch using only affine maps and
non-linearities. We will also see how to compute a loss function, using
PyTorch's built in negative log likelihood, and update parameters by
backpropagation.

All network components should inherit from nn.Module and override the
forward() method. That is about it, as far as the boilerplate is
concerned. Inheriting from nn.Module provides functionality to your
component. For example, it makes it keep track of its trainable
parameters, you can swap it between CPU and GPU with the ``.to(device)``
method, where device can be a CPU device ``torch.device("cpu")`` or CUDA
device ``torch.device("cuda:0")``.

Let's write an annotated example of a network that takes in a sparse
bag-of-words representation and outputs a probability distribution over
two labels: "English" and "Spanish". This model is just logistic
regression.




# Example: Logistic Regression Bag-of-Words classifier: 

Our model will map a sparse BoW representation to log probabilities over
labels. We assign each word in the vocab an index. For example, say our
entire vocab is two words "hello" and "world", with indices 0 and 1
respectively. The BoW vector for the sentence "hello hello hello hello"
is

\begin{align}\left[ 4, 0 \right]\end{align}

For "hello world world hello", it is

\begin{align}\left[ 2, 2 \right]\end{align}

etc. In general, it is

\begin{align}\left[ \text{Count}(\text{hello}), \text{Count}(\text{world}) \right]\end{align}

Denote this BOW vector as $x$. The output of our network is:

\begin{align}\text{Sigmoid}(Ax + b)\end{align}

That is, we pass the input through an affine map and then do sigmoid.




这段内容介绍了**逻辑回归（Logistic Regression）如何应用于袋-of-词（Bag-of-Words, BoW）文本分类**。以下是逐步解析：

---

### **1. 介绍 BoW 词袋模型**
BoW 是一种简单的文本表示方法，它**不考虑词语的顺序**，而是**仅仅统计每个词在文本中出现的次数**。

例如：
- 词汇表（vocabulary）中只有两个单词："hello" 和 "world"。
- 我们为这两个单词分配索引："hello" 作为索引 0，"world" 作为索引 1。

---

### **2. 计算 BoW 向量**
**示例 1：**
对于句子 **"hello hello hello hello"**，它的 BoW 表示是：
\[
[4, 0]
\]
- "hello" 出现 4 次，对应索引 0，值为 4。
- "world" 没出现，对应索引 1，值为 0。

**示例 2：**
对于句子 **"hello world world hello"**，它的 BoW 表示是：
\[
[2, 2]
\]
- "hello" 出现 2 次，对应索引 0，值为 2。
- "world" 也出现 2 次，对应索引 1，值为 2。

**一般公式：**
\[
[\text{Count(hello)}, \text{Count(world)}]
\]
即，BoW 向量的每个元素表示词汇表中相应单词在句子中出现的次数。

---

### **3. 逻辑回归模型**
记 BoW 向量为 \( x \)，那么我们的**神经网络输出**为：
\[
\text{Sigmoid}(Ax + b)
\]

这里：
- \( A \) 是一个权重矩阵。
- \( b \) 是偏置项。
- 这意味着，我们首先对输入 \( x \) 进行一个**线性变换（仿射变换）**：
  \[
  Ax + b
  \]
  然后将结果传递到**Sigmoid** 函数中，得到最终的**概率输出**。

---

### **4. 逻辑回归的作用**
- **Sigmoid 函数**将线性变换的结果映射到 \([0,1]\)，可以解释为**属于某个类别的概率**。
- **如果输出接近 1**，意味着模型认为该文本更可能属于某个类别（比如 "positive sentiment"）。
- **如果输出接近 0**，意味着该文本更可能属于另一个类别（比如 "negative sentiment"）。

---

### **5. 结论**
- **BoW 仅统计词频**，不考虑单词顺序，因此是一个**简单但有效**的文本表示方式。
- **逻辑回归**在 BoW 上应用，通过学习权重 \( A \) 和偏置 \( b \) 来进行分类。
- **Sigmoid 函数**用于输出概率，从而进行二元分类（如情感分析）。

如果你有更具体的疑问，欢迎继续讨论！ 😊

In [7]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
#这个词典可以用于构建 Bag-of-Words（BoW）表示，即将每个句子转换为一个向量，该向量的每个索引对应一个单词的出现次数。
word_to_ix = {}
for sent, _ in data + test_data: #_（下划线）是一个占位符变量，表示我们不会使用这个值。
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [8]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))    # shape: [26] ; vector
    for word in sentence:
        vec[word_to_ix[word]] += 1

    #.view(1, -1) 将 vec 变成 1 x N 形状的矩阵（行向量），方便神经网络处理：
    return vec.view(1, -1)    # return a matrix: 1 x len(vec)

def make_target(label, label_to_ix):
    #将文本标签（如 "SPANISH" 或 "ENGLISH"）转换为 数值标签（0 或 1）
    return torch.LongTensor([label_to_ix[label]])  # [0] or [1]

In [9]:
bow_vector = make_bow_vector("me gusta comer en la cafeteria".split(), word_to_ix)
print(bow_vector)
print(bow_vector.shape) # matrix

tensor([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([1, 26])


In [10]:
target = make_target("SPANISH", label_to_ix)
print(target)  
print(target.shape) # vector

tensor([0])
torch.Size([1])


In [11]:
print(target.float().unsqueeze(dim=0)) # matrix  ; convert size([1]) to size([1,1])
print(target.float().unsqueeze(dim=0).shape)

tensor([[0.]])
torch.Size([1, 1])


In [12]:
print(target.float().view(1,-1)) # matrix ; convert size([1]) to size([1,1])
print(target.float().view(1,-1).shape)

tensor([[0.]])
torch.Size([1, 1])


In [13]:
VOCAB_SIZE = len(word_to_ix)   # 26 in this example
OUTPUT_SIZE = 1

class BoWClassifier(nn.Module):  # inheriting from nn.Module!

    def __init__(self, vocab_size, output_size):
        # calls the init function of nn.Module.  Dont get confused by syntax,
        # just always do it in an nn.Module
        super(BoWClassifier, self).__init__()

        # Define the parameters that you will need.  In this case, we need A and b,
        # the parameters of the affine mapping.
        # Torch defines nn.Linear(), which provides the affine map.
        # Make sure you understand why the input dimension is vocab_size
        # and the output is output_size!
        
        self.linear = nn.Linear(vocab_size, output_size)  # output_size = 1

        # NOTE! The non-linearity sigmoid does not have parameters! So we don't need
        # to worry about that here

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through sigmoid.
        # Many non-linearities and other functions are in torch.nn.functional
        
        return torch.sigmoid(self.linear(bow_vec))  # use sigmoid

model = BoWClassifier(VOCAB_SIZE, OUTPUT_SIZE)

# the model knows its parameters.  The first output below is A, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the PyTorch devs, your module
# (in this case, BoWClassifier) will store knowledge of the nn.Linear's parameters

for param in model.parameters():
    print(param, "; shape: ", param.shape)


Parameter containing:
tensor([[ 0.1714,  0.0610, -0.0730, -0.1184, -0.0329, -0.0846, -0.0628,  0.0094,
          0.1169,  0.1066, -0.1917,  0.1216,  0.0548,  0.1860,  0.1294, -0.1787,
         -0.1865, -0.0946,  0.1722, -0.0327,  0.0839, -0.0911,  0.1924, -0.0830,
          0.1471,  0.0023]], requires_grad=True) ; shape:  torch.Size([1, 26])
Parameter containing:
tensor([-0.1033], requires_grad=True) ; shape:  torch.Size([1])


In [14]:
# To run the model, pass in a BoW vector
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    sample = data[0]                                      # ([me, gusta, comer, en, la, cafeteria], "SPANISH")
    bow_vector = make_bow_vector(sample[0], word_to_ix)  
    output_probs = model(bow_vector)
    print(output_probs)   # i.e. probablity of English class

tensor([[0.4552]])


In [15]:
# Run on test data before we train, just to see a before-and-after
with torch.no_grad():
    for instance, label in test_data:                   # first example:  ([Yo, creo, que, si], "SPANISH")  
                                                        # second example: ([it, is, lost, on, me], "ENGLISH")
        bow_vec = make_bow_vector(instance, word_to_ix)
        output_probs = model(bow_vec)
        print(output_probs)

# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([[0.4727]])
tensor([[0.4507]])
tensor([-0.1917], grad_fn=<SelectBackward0>)


So lets train! To do this, we pass instances through to get 
probabilities, compute a loss function, compute the gradient of the loss
function, and then update the parameters with a gradient step. Loss
functions are provided by Torch in the nn package. <b>nn.BCELoss() is the
Binary Cross Entropy between the target and the output.</b> It also defines optimization
functions in torch.optim. Here, we will just use SGD.

In [16]:
loss_function = nn.BCELoss()   # use BCELoss() to measures the Binary Cross Entropy between the target and the output
                               # use CrossEntropyLoss for multi-class classification, such as 0, 1, 2, or 3
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Tensor as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. 
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix).float().view(1,-1) # make type and dimension (1x1) right                                                      
        #print(target)

        # Step 3. Run our forward pass.
        output_probs = model(bow_vec)
        #print(output_probs)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(output_probs, target)   # target has the same shape as the output_probs
        loss.backward()
        optimizer.step()

with torch.no_grad():
    for instance, label in test_data:  # first example:  ([Yo, creo, que, si], "SPANISH")  
                                       # second example: ([it, is, lost, on, me], "ENGLISH")
        bow_vec = make_bow_vector(instance, word_to_ix)
        output_probs = model(bow_vec)
        if output_probs.item() < 0.5:
            print(instance, ": ", output_probs.item(), ": SPANISH")
        else:
            print(instance, ": ", output_probs.item(), ": ENGLISH")

# Weight for "creo" goes down!
print(next(model.parameters())[:, word_to_ix["creo"]])

['Yo', 'creo', 'que', 'si'] :  0.14345665276050568 : SPANISH
['it', 'is', 'lost', 'on', 'me'] :  0.8789470195770264 : ENGLISH
tensor([-0.9463], grad_fn=<SelectBackward0>)


We got the right answer! You can see that the probability for English is lower than 0.5 in the first example, and the probability for English is much higher than 0.5 in the second for the test data, as it should be.

In [17]:
for i, param in enumerate(model.parameters()):
    if i == 0:
        for key, value in word_to_ix.items():
            print(key, ": ", param[0][value].item())
    else:
        print("\nb: ", param.item()) # Use torch.Tensor.item() to get a Python number from a tensor containing a single value

me :  0.31328997015953064
gusta :  -0.6709371209144592
comer :  -0.805001437664032
en :  -0.8504132032394409
la :  -0.764838695526123
cafeteria :  -0.8165594339370728
Give :  0.8109734654426575
it :  1.327536940574646
to :  1.4350569248199463
No :  -0.20364010334014893
creo :  -0.946277916431427
que :  -0.632987916469574
sea :  -0.25544866919517517
una :  -0.5685372352600098
buena :  -0.6251159906387329
idea :  -0.4889252483844757
is :  0.25785887241363525
not :  0.3497346043586731
a :  0.6165398359298706
good :  0.4116608798503876
get :  0.5282576084136963
lost :  0.3531903028488159
at :  0.6367605924606323
Yo :  -0.08297643065452576
si :  0.14707234501838684
on :  0.002322339452803135

b:  -0.2717020809650421


# Exercise:

1. Build a simple sentiment classifier with a new data given below, and predict test_data: utilize BoWClassifier() defined in previous steps.
1. 使用下面给出的新数据构建一个简单的情绪分类器，并预测测试数据：利用前面步骤中定义的 BoWClassifier()。

In [18]:
data = [("well done".split(), "positive"),
        ("good work".split(), "positive"),
        ("great effort".split(), "positive"),
        ("weak".split(), "negative"),
        ("poor effort".split(), "negative"),
        ("not good".split(), "negative")]

test_data = [("great work".split(), "positive"),
             ("good job".split(), "positive"),
             ("poor work".split(), "negative"),
             ("not great".split(), "negative")]

In [19]:
# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

label_to_ix = {"negative": 0, "positive": 1}

{'well': 0, 'done': 1, 'good': 2, 'work': 3, 'great': 4, 'effort': 5, 'weak': 6, 'poor': 7, 'not': 8, 'job': 9}
